In [8]:
import pandas as pd
import numpy as np
from ts_fe import timeseries_processing as tsp

df = pd.DataFrame()

df["A"] = np.random.normal(0, 1, size=100)
df["B"] = np.random.normal(0, 5, size=100)
df["C"] = np.random.normal(0, 10, size=100)

In [2]:
lag_df = tsp.lag_column(df, 5, "C")

In [9]:
df.shape, lag_df.shape

((100, 3), (95, 8))

In [3]:
lag_df.head()

,A,B,C,C_lagged_1,C_lagged_2,C_lagged_3,C_lagged_4,C_lagged_5
5,0.329895,2.621327,-8.791425,-12.613909,5.674264,-8.077809,-1.766799,-3.741372
6,0.396971,3.153765,-2.152028,-8.791425,-12.613909,5.674264,-8.077809,-1.766799
7,-0.462153,-9.477816,-21.792292,-2.152028,-8.791425,-12.613909,5.674264,-8.077809
8,-0.980066,6.416591,-13.964037,-21.792292,-2.152028,-8.791425,-12.613909,5.674264
9,0.553141,16.122892,-10.492030,-13.964037,-21.792292,-2.152028,-8.791425,-12.613909


In [4]:
cols = lag_df.columns.tolist()
cols.remove("C")
y = lag_df["C"]
X = lag_df[cols]
X_train, X_test, y_train, y_test = tsp.timeseries_train_test_split(X, y, position=0.5)

In [8]:
X_train.shape

(47, 7)

In [12]:
X_train.head()

,A,B,C_lagged_1,C_lagged_2,C_lagged_3,C_lagged_4,C_lagged_5
5,0.021964,-0.910783,11.031396,8.416277,1.124477,-16.064397,-0.730463
6,-0.818135,-1.531578,-9.597114,11.031396,8.416277,1.124477,-16.064397
7,-0.496949,-4.120583,-0.680343,-9.597114,11.031396,8.416277,1.124477
8,0.548241,1.791687,7.474070,-0.680343,-9.597114,11.031396,8.416277
9,-1.446739,-1.628268,-8.182607,7.474070,-0.680343,-9.597114,11.031396


In [16]:
X_train_new = X_train.values.reshape(
    (X_train.shape[0], 1, X_train.shape[1])
)

In [18]:
X_train_new.shape

(47, 1, 7)

In [5]:
lstm_X_train, lstm_X_test = tsp.lstm_dataprep(X_train, X_test)

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mean_squared_error(y_test, y_pred)

200.18881490446674

In [10]:
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import matplotlib.pyplot as plt

look_back = 5 + 2
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(lstm_X_train, y_train, epochs=100, batch_size=1, verbose=2)

# make predictions
trainPredict = model.predict(lstm_X_train)
testPredict = model.predict(lstm_X_test)

trainScore = math.sqrt(mean_squared_error(y_train[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

Epoch 1/100
47/47 - 14s - loss: 107.3994
Epoch 2/100
47/47 - 0s - loss: 106.8225
Epoch 3/100
47/47 - 0s - loss: 106.4073
Epoch 4/100
47/47 - 0s - loss: 105.8626
Epoch 5/100
47/47 - 0s - loss: 105.1871
Epoch 6/100
47/47 - 0s - loss: 104.5789
Epoch 7/100
47/47 - 0s - loss: 104.1870
Epoch 8/100
47/47 - 0s - loss: 103.7996
Epoch 9/100
47/47 - 0s - loss: 103.3767
Epoch 10/100
47/47 - 0s - loss: 103.0047
Epoch 11/100
47/47 - 0s - loss: 102.6852
Epoch 12/100
47/47 - 0s - loss: 102.4108
Epoch 13/100
47/47 - 0s - loss: 102.1555
Epoch 14/100
47/47 - 0s - loss: 101.8539
Epoch 15/100
47/47 - 0s - loss: 101.5793
Epoch 16/100
47/47 - 0s - loss: 101.2990
Epoch 17/100
47/47 - 0s - loss: 101.0286
Epoch 18/100
47/47 - 0s - loss: 100.7845
Epoch 19/100
47/47 - 0s - loss: 100.5379
Epoch 20/100
47/47 - 0s - loss: 100.2854
Epoch 21/100
47/47 - 0s - loss: 100.0527
Epoch 22/100
47/47 - 0s - loss: 99.8056
Epoch 23/100
47/47 - 0s - loss: 99.5752
Epoch 24/100
47/47 - 0s - loss: 99.2078
Epoch 25/100
47/47 - 0s - l

NameError: name 'mean_squared_error' is not defined